In [1]:
import psycopg2

In [2]:
#creating a connection to the database
try:
    conn = psycopg2.connect(database="postgres", user="postgres", password="Meena@4444", host="localhost", port="5433")
except psycopg2.Error as e:
        print("Error:Could not make connection to the PG database")
        print(e)
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error:Could not get cursor to the DB")
    print(e)
conn.set_session(autocommit=True)

#### now lets Imagine we work at an online music store. there will be many tables in our database, but lets just focus on 4 tables around customer purhases. 

In [3]:
# lets do this step by step 
# step 1 : Creating the fact table and inser the data into the table
try: 
    cur.execute('''
    CREATE TABLE customer_transactions
    (customer_id int, store_id int, amount_spent float)                
    ''')
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
#Insert into all tables 
try: 
    cur.execute('''
    INSERT INTO customer_transactions
    (customer_id, store_id, amount_spent)
    VALUES (%s, %s, %s)
    ''', (1, 1, 20.5))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
try: 
    cur.execute('''
    INSERT INTO customer_transactions
    (customer_id, store_id, amount_spent)
    VALUES (%s, %s, %s)
    ''', (2, 1, 35.21))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

In [4]:
#step2: Creating the dimansion table and insrerting the data into the table
try: 
    cur.execute('''
    CREATE TABLE customer
    (customer_id int, name varchar, rewards varchar)                
    ''')
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
try: 
    cur.execute('''
    INSERT INTO customer
    (customer_id, name, rewards) 
    VALUES (%s, %s, %s)
    ''', (1, 'Amanda', 'Y'))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute('''
    INSERT INTO customer
    (customer_id, name, rewards)
    VALUES (%s, %s, %s)
    ''', (2, 'Toby', 'N'))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute('''
    CREATE TABLE items_purchased
    (customer_id int, item_number int, item_name varchar)                
    ''')
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
try: 
    cur.execute('''
    INSERT INTO items_purchased
    (customer_id, item_number, item_name)
    VALUES (%s, %s, %s)
    ''', (1, 1, 'Rubber Soul'))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
try: 
    cur.execute('''
    INSERT INTO items_purchased
    (customer_id, item_number, item_name)
    VALUES (%s, %s, %s)
    ''', (2, 3, 'Let It Be'))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute('''
    CREATE TABLE store
    (store_id int, state varchar)                
    ''')
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
try: 
    cur.execute('''
    INSERT INTO store
    (store_id, state)
    VALUES (%s, %s)
    ''', (1, 'CA'))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

try: 
    cur.execute('''
    INSERT INTO store
    (store_id, state)
    VALUES (%s, %s)
    ''', (2, 'WA'))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

### No lets run few queries on this data. 
1. Find all the customers that spent more than 30 dollars, who are they, which store they bought it from, location of the store, what they bought and if they are a rewards member.


In [5]:
try: 
    cur.execute('''
    SELECT customer.name, store.store_id, store.state,
    items_purchased.item_name, customer.rewards
    FROM customer_transactions
    JOIN customer ON customer_transactions.customer_id = customer.customer_id
    JOIN store ON customer_transactions.store_id = store.store_id
    JOIN items_purchased ON customer_transactions.customer_id = items_purchased.customer_id
    WHERE customer_transactions.amount_spent > 30
    ''')
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

('Toby', 1, 'CA', 'Let It Be', 'N')


2. How much did Customer 2 spend?

In [6]:
try: 
    cur.execute('''
    SELECT customer_id, SUM(amount_spent) FROM customer_transactions
    WHERE customer_id=2
    GROUP BY customer_id
    ''')
    
    
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

(2, 35.21)


In [7]:
# drop the tables 
try: 
    cur.execute("DROP TABLE customer_transactions")
    cur.execute("DROP TABLE customer")
    cur.execute("DROP TABLE store")
    cur.execute("DROP TABLE items_purchased")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)

In [8]:
cur.close()
conn.close()